In [ ]:
import pandas as pd
import numpy as np
import glob
import os

In [ ]:
TARGET_SUBREDDITS = [
    'localllama', 'characterai', 'sillytavernai', 'janitorai_official',
    'myboyfriendisai', 'chatgpt', 'artificialinteligence', 'singularity',
    'fanfiction', 'lonely'
]
TARGET_SUBREDDITS_LOWER = [s.lower() for s in TARGET_SUBREDDITS]

DATA_DIRECTORY = 'filtered_data'

In [ ]:
def clean_submissions_df(df_raw):

    # considering that the data that will undergo this preprocessing has
    # already been filtered by the necessary subreddits using reddit_zst_filter,
    # this is more of an additional security measure in case of unexpected situations
    df_filtered = df_raw[df_raw['subreddit'].str.lower().isin(TARGET_SUBREDDITS_LOWER)].copy()

    essential_rs_cols = [
        'id',              # unique id for the submission
        'created_utc',     # timestamp
        'author',          # user who posted
        'subreddit',       # subreddit it was posted in
        'title',           # the title of the post
        'selftext',        # the body text (if a self-post)
        'score',           # upvotes/downvotes
        'num_comments',    # how many comments it sparked
        'over_18',         # NSFW check is relevant to our topic
        'url',             # url if it's a link post
        'permalink'        # permanent link to the thread
    ]

    cols_to_keep = [col for col in essential_rs_cols if col in df_filtered.columns]
    df_filtered = df_filtered[cols_to_keep]

    df_filtered = df_filtered.drop_duplicates(subset=['id']) # duplicates

    # normalization, converting UNIX timestamp to a datetime object
    df_filtered['created_utc'] = pd.to_datetime(df_filtered['created_utc'], unit='s')

    # normalization of common null/deleted text values to numpy.NaN
    # this ensures '[removed]' isn't counted as a word in text analysis
    null_values = ['[removed]', '[deleted]', '']
    df_filtered['selftext'] = df_filtered['selftext'].replace(null_values, np.nan)

    # normalization for deleted authors
    df_filtered['author'] = df_filtered['author'].replace('[deleted]', np.nan)

    return df_filtered

In [ ]:
def clean_comments_df(df_raw):

    df_filtered = df_raw[df_raw['subreddit'].str.lower().isin(TARGET_SUBREDDITS_LOWER)].copy()

    essential_rc_cols = [
        'id',              # unique id
        'link_id',         # id of the parent submission (links to RS 'id')
        'parent_id',       # id of the parent comment (for threading)
        'created_utc',     # timestamp
        'author',          # user who commented
        'subreddit',       # subreddit
        'body',            # the text of the comment
        'score'            # upvotes/downvotes
    ]

    cols_to_keep = [col for col in essential_rc_cols if col in df_filtered.columns]
    df_filtered = df_filtered[cols_to_keep]

    # duplicates
    df_filtered = df_filtered.drop_duplicates(subset=['id'])

    # normalization
    df_filtered['created_utc'] = pd.to_datetime(df_filtered['created_utc'], unit='s')

    null_values = ['[removed]', '[deleted]', '']
    df_filtered['body'] = df_filtered['body'].replace(null_values, np.nan)

    df_filtered['author'] = df_filtered['author'].replace('[deleted]', np.nan)

    # pushshift data has prefixes like 't3_' (submission) or 't1_' (comment)
    # we remove these to get the clean ID for merging
    df_filtered['link_id'] = df_filtered['link_id'].str.replace('t3_', '')
    df_filtered['parent_id'] = df_filtered['parent_id'].str.replace(r't[13]_', '', regex=True)

    return df_filtered

In [ ]:
#  1. process all RS files
rs_files = glob.glob(os.path.join(DATA_DIRECTORY, "RS_*.parquet"))
all_rs_dfs = []

print(f"\nFound {len(rs_files)} RS files")
for f in rs_files:
    print(f"  Processing {f}")
    try:
        df_raw = pd.read_parquet(f)
        df_clean = clean_submissions_df(df_raw)
        all_rs_dfs.append(df_clean)
        print(f"    -> Found {len(df_clean)} relevant submissions")
    except Exception as e:
        print(f"    ERROR processing {f}: {e}")

# 1.1. combine into one DF
final_rs_df = pd.concat(all_rs_dfs, ignore_index=True)
final_rs_df = final_rs_df.drop_duplicates(subset=['id']) # final de-dupe in case a post appears in two monthly files

print(f"Shape: {final_rs_df.shape}")
print(final_rs_df.info())

final_rs_df.to_parquet('all_submissions_cleaned.parquet', index=False)

In [ ]:
# 2. process all RC files
rc_files = glob.glob(os.path.join(DATA_DIRECTORY, "RC_*.parquet"))
all_rc_dfs = []

print(f"\nFound {len(rc_files)} RC files")
for f in rc_files:
    print(f"  Processing {f}")
    try:
        df_raw = pd.read_parquet(f)
        df_clean = clean_comments_df(df_raw)
        all_rc_dfs.append(df_clean)
        print(f"    -> Found {len(df_clean)} relevant comments")
    except Exception as e:
        print(f"    ERROR processing {f}: {e}")

# 2.1. combine into one DF
final_rc_df = pd.concat(all_rc_dfs, ignore_index=True)
final_rc_df = final_rc_df.drop_duplicates(subset=['id'])

print(f"Shape: {final_rc_df.shape}")
print(final_rc_df.info())

final_rc_df.to_parquet('all_comments_cleaned.parquet', index=False)